In [ ]:
import dlib
import time
import os

from PIL import Image
from keras.layers import Activation,K
from keras.models import load_model
from keras.utils import plot_model, get_custom_objects, np_utils
import cv2
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
import gc
import itertools
import numpy as np
import matplotlib.pyplot as plt
def cal_acc(name, accuracy):
    print("{} Accuracy = {:.2f}% - (+/- {:.2f}%)".format(name, np.mean(accuracy), np.std(accuracy)))

def get_data_from_path(path = 'D:\\NianXiang_File\\CNN\\Dataset\\LE', file_name='BE'):
    # data = np.load('predict_y_nor.npy')
    output = {}
    x = []
    y = []
    folders = []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in dirs:
            folders.append(name)
    folders = folders[-7:]
    start_time = time.time()
    count = 0
    for fld in folders:
        index = folders.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        for root, dirs, files in os.walk(path + '\\' + fld, topdown=False):
            for file in files:
                print(file)
                if (file == '1.jpg'):
                    pass
                else:
                    filepath = root + '\\' + file
                    detector = dlib.get_frontal_face_detector()
                    img = cv2.imread(filepath)
                    face = detector(img, 1)
                    for i, d in enumerate(face):
                        x1 = d.left()
                        y1 = d.top()
                        x2 = d.right()
                        y2 = d.bottom()
                        img = img[y1:y2, x1:x2]
                    lpath = 'D:\\NianXiang_File\\CNN\\Dataset\\small_dataset\\NEW'
                    filename = lpath + '\\' + str(index) + '\\' + str(count) + '.jpg'
                    cv2.imwrite(filename, img)
                    try:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        resized = cv2.resize(img, (224, 224), cv2.INTER_LANCZOS4)
                        y.append(index)
                        x.append(resized)
                        count = count + 1
                    except:
                        pass

                # file = Data.static_load_image(ffile)
    np.save('{}_X.npy'.format(file_name), x)
    np.save('{}_Y.npy'.format(file_name), y)
    print("Complete")

def confusion_matri(model=None, normalize=False, title='Confusion Matrix', cls='be', x=None, y=None):
    predict = model.predict(x=x, batch_size=16, verbose=2)
    pred = np.argmax(predict, axis=1)
    loss, score = model.evaluate(x=x, y=y, batch_size=16, verbose=2)

    cnf_matrix = confusion_matrix(np.argmax(y, axis=1), pred)
    np.set_printoptions(precision=2)
    plt.figure()
    if cls == 'be':
        class_name = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    if cls == 'le':
        class_name = ["Frustration", "Confused", "Bored", "Delightful", "Flow", "Surprise"]
    __plot_confusion_matrix(cnf_matrix, classes=class_name,
                            title=title,
                            normalize=normalize)
    print('{:.2f}%'.format(score * 100))
    plt.show()
    # print("Accuracy：{:.2f}%".format(score * 100))

def __plot_confusion_matrix(cm, classes,
                            normalize=False,
                            title='Confusion matrix',
                            cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
if __name__=="__main__":
    def swish(x):
        return K.sigmoid(x) * x


    class Swish(Activation):
        def __init__(self, activation, **kwargs):
            super(Swish, self).__init__(activation, **kwargs)
            self.__name__ = 'swish'
    get_custom_objects().update({"swish": Swish(swish)})
    #
    # path = "D:\\NianXiang_File\\CNN\\Dataset\\mix\\"
    #
    model = load_model('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Train\\callbacks.26-0.84091.hdf5')

    # for i in range(7):
    #     x = np.load('C_FRONT_X_{}.npy'.format(i))
    #     # y = np_utils.to_categorical(y, 7).astype('int')
    #     x = x.reshape(x.shape[0], 224, 224, 1)
    #     predict = model.predict(x=x, batch_size=16, verbose=2)
    #     pred = np.argmax(predict, axis=1)
    #     print("{} - {}".format(i, pred))
    #
    #     for index, img in enumerate(x):
    #         single_path = path + "{}".format(pred[index])
    #         if not os.path.exists(single_path):
    #             os.makedirs(single_path)
    #         img = cv2.imwrite(single_path + '\\{}.jpg'.format(index), img)
    # loss, score = model.evaluate(x=x, y=y, batch_size=16, verbose=2)
    # x = np.load('data/Model2_X.npy')
    # x = np.load('DATA_TEST_X.npy')
    # y = np.load('data/Model2_Y.npy')
    # x = np.load('C_FINAL_X.npy')
    # y = np.load('DATA_TEST_Y.npy')
    # print(np.unique(y))
    x = np.load('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Dataset\\s_data_x.npy')
    y = np.load('E:\\交接\\1_賴念祥code\\賴念祥code\\Researchyes\\Research\\Dataset\\s_data_y.npy')
    y = np_utils.to_categorical(y, 6).astype('int')
    x = x.reshape(x.shape[0], 224, 224, 1)
    print(x.shape, y.shape)
    # loss, score = model.evaluate(x=x, y=y, batch_size=16, verbose=2)
    # print(score)

    # pred = model.predict(x=x,batch_size=64,verbose=1)
    # pred = np.argmax(pred, axis=1)
    # print(pred.shape)
    confusion_matri(model=model, normalize=False,
                    title="Learning Emotion",
                    cls='le',x=x,y=y)
        # for index, data in enumerate(predict):
        # predict[index] = np.argmax(data)
    # np.save('outcome.npy', predict)

    # model = load_model('D:\\NianXiang_File\\CNN\\Outcome\\LearningEmotion\\NXNet\\transfer learning\\NXNet_LE.17-0.85572.hdf5')
    # x1 = np.load('4106029006_X.npy')
    # x2 = np.load('4104030407_X.npy')
    # x3 = np.load('4103044033_X.npy')
    # x1 = np.load('C_X.npy')
    # y1 = np.load('C_Y.npy')
    # x2 = np.load('F_X.npy')
    # y2 = np.load('F_Y.npy')
    # x3 = np.load('NEW_LE_X.npy')
    # y3 = np.load('NEW_LE_Y.npy')
    # X = np.load('combine_le_x1.npy')
    # Y = np.load('combine_le_y1.npy')
    # x = np.load('KDEF_x.npy')
    # y = np.load("KDEF_y.npy")
    # _x = []
    # _y = []
    # for index, label in enumerate(y):
    #     if label == 3:
    #        _x.append(x[index])
    #        _y.append(label)
    # _x = np.array(_x)
    # _y = np.array(_y)
    # Y = np.array([Y, _y])
    # yy = []
    # for i in Y:
    #     for data in i:
    #       yy.append(data)
    # Y = np.array(yy)
    #
    # X = np.vstack((X, _x))
    #
    # print(X.shape, Y.shape)
    # np.save('combine_le_argument_x.npy', X)
    # np.save('combine_le_argument_y.npy', Y)
    # y = np.load('KDEF_y.npy')
    # count0 ,count1, count2, count3, count4 ,count5 ,count6 = 0,0,0,0,0,0,0
    # for data in y:
    #     data = int(data)
    #     if data == 0:
    #         count0 += 1
    #     if data == 1:
    #         count1 += 1
    #     if data == 2:
    #         count2 += 1
    #     if data == 3:
    #         count3 += 1
    #     if data == 4:
    #         count4 += 1
    #     if data == 5:
    #         count5 += 1
    #     if data == 6:
    #         count6 += 1
    # print(count0 ,count1, count2, count3, count4 ,count5 ,count6)
    # x = np.vstack((x1, x2))
    # Y = np.array([y1, y2 ])
    # y = []
    # for i in Y:
    #     for data in i:
    #       y.append(data)
    # y = np.array(y)
    # # y = np.vstack((y, y1))
    # np.save('combine_le_x1.npy', x)
    # np.save('combine_le_y1.npy', y)

    # x = x.reshape(x.shape[0], 224, 224, 1)
    # y = np_utils.to_categorical(y, 7).astype('int')
    # confusion_matri(model=model, normalize=False, title="Learning Emotion",cls='le',x=x,y=y)
    # datas = []
    # for data in output:
    #     datas.append(np.argmax(data))
    # np.save('0_Y.npy'.format(), datas)
    # input_list = [x1, x2, x3, x4]
    # for index, x in enumerate(input_list):
    #     x = x.reshape(x.shape[0], 224, 224, 1)
    #     output = model.predict(x=x, batch_size=64, verbose=2)
    #     datas = []
    #     for data in output:
    #         datas.append(np.argmax(data))
    #     np.save('{}_Y.npy'.format(index), datas)
    # np.save('output.npy', y)



    # folders = []
    # path = 'D:\\NianXiang_File\\CNN\\Dataset\\LE_Test\\Final'
    # for root, dirs, files in os.walk(path, topdown=False):
    #     for name in dirs:
    #         folders.append(name)
    # folders = folders[-7:]
    # y_list = [2,1,0,3]
    # x = []
    # start_time = time.time()
    # count = 0
    # y = np.load('outcome.npy')
    # for fld in folders:
    #     index = folders.index(fld)
    #     y = np.load('{}_Y.npy'.format(index))
    #     print('Loading {} files (Index: {})'.format(fld, index))
    #     fold = os.path.basename(fld)
    #     p = os.path.join(path, fold, '*g')
    #     files = glob.glob(p)
    #     output = []
    #     x = []
    #     for i, fl in enumerate(files):
    #         flbase = os.path.basename(fl)
    #         data = np.array([str(flbase).split('.')[0], y[i]])
    #         output.append(data)
    #     import pandas as pd
    #     p = pd.DataFrame(data=output, columns=['file', 'class'])
    #     p.to_csv('{}.csv'.format(fld))

    #         img = cv2.imread(fl)
    #         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #         data_x = cv2.resize(img, (224, 224), cv2.INTER_LANCZOS4)
    #         x.append(data_x)
    #     np.save('{}_X.npy'.format(fld), x)
    #     x = []


    # np.save('NEW_LE_Y.npy', y)
    # get_data_from_path(path='D:\\NianXiang_File\\CNN\\Dataset\\small_dataset\\ORIGIN', file_name='small_data')
    # y = np.load('new_le_train_y.npy')
    # print(len(y))
    # # x = np.load("NEW_LE_X.npy")
    # y = np.load("NEW_LE_Y.npy")
    # print(len(y))
    # X = []
    # Y = []
    # for i in range(len(y)):
    #     if y[i] == 4 or y[i] == 6:
    #         x[i] = np.zeros((224,224))
    #     X.append(x[i])
    #     Y.append(y[i])
    # np.save('NEW_LE_X_1.npy', X)
    # np.save('NEW_LE_Y_1.npy', Y)

    # path = 'D:\\NianXiang_File\\CNN\\Dataset\\NEW_LE\\'
    # index = 0
    # for i in range(len(y)):
    #     filename = path + str(y[i]) + '\\' + str(index) + '.jpg'
    #     cv2.imwrite(filename, x[i])
    #     print(filename)
    #     index += 1
    # import tensorflow as tf
    # import keras.backend.tensorflow_backend as KTF
    #
    # KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu': 0})))
    # model = load_model('NXNet_fold3_LE.11-0.79500.hdf5')
    # plot_model(model, to_file='NXNet.png')
    # cal_acc('DenseNet', [97.77, 97.67, 95.23, 95.23, 90.24])
    # cal_acc('Inception-ResNetV2', [92.68, 90.47, 78.57, 88.37, 97.77])
    # cal_acc('InceptionV3', [86.66, 79.07, 78.57, 85.71, 82.92])
    # cal_acc('VGG', [82.22, 93.02, 78.57, 85.71, 92.68])
    # cal_acc('NXNet', [95.55, 95.34, 90.47, 85.71, 87.80])
    #
    # # x1 = np.load('JAFFE_x.npy')
    # x2 = np.load('KDEF_x.npy')
    # y1 = np.load('JAFFE_y.npy')
    # y2 = np.load('KDEF_y.npy')
    # x = np.concatenate((x1, x2), axis=0)
    # y = np.concatenate((y1, y2), axis=0)
    # np.save('KDEF_JAFFE_x.npy', x)
    # np.save('KDEF_JAFFE_y.npy', y)
    # print('have {} data'.format(len(y)))
    # Dataset.load_jaffe()
    # Dataset.load_KDEF_dataset()